# Code for Assessing Privacy Risk Using the Attack from Ponte et al. (2024)

https://github.com/GilianPonte/whereswaldoIJRM

In [64]:
import pandas as pd
import numpy as np
import csv
import os
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
%matplotlib inline

# Add the parent directory to path
sys.path.append('..')

# Then import
from helper_functions import *

Import a subset of the Criteo dataset.

## Note

If you are synthesizing variables out of the order in which they appear in the data, you need to re-order the initial training data to match that order.

## End Note

In [65]:
train_data = pd.read_csv("../../Data/Criteo/cleaned_criteo_small.gz",
                         compression='gzip', 
                         sep='\,',
                         header=0,
                         engine='python')

Drop duplicates and reset index.

In [66]:
train_data = pd.DataFrame.drop_duplicates(train_data)
# reset dataframe index
train_data.reset_index(drop=True, inplace=True)
train_data

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
0,2.954807,2.368327,2.178833,1.480282,2.330251,20.349663,0.000205,2.100938,48.158740,2.579463,1.667778,0.84478,1,0,0,0
1,3.100481,2.308533,2.105887,107.757316,2.330251,61.279994,0.275765,1.575636,53.083067,2.579463,1.667778,0.84478,1,0,0,0
2,2.945821,2.308533,2.139045,9.914112,2.330251,61.279994,0.003786,1.575636,46.910867,3.486706,1.667778,0.84478,0,0,0,0
3,3.177230,2.308533,2.105887,107.757316,2.330251,61.279994,0.089715,1.575636,53.083067,2.579463,1.667778,0.84478,0,0,0,0
4,3.274234,2.308533,2.105887,107.757316,2.330251,61.279994,0.000003,1.575636,53.083067,2.579463,1.667778,0.84478,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99904,3.037003,2.308533,2.184994,107.757316,2.330251,61.279994,0.275765,1.575636,47.003884,2.579463,1.667778,0.84478,1,0,0,0
99905,3.286020,2.308533,2.105887,107.757316,2.330251,61.279994,0.275765,1.575636,53.083067,2.579463,1.667778,0.84478,1,0,0,0
99906,3.285632,2.308533,2.105887,107.757316,2.330251,61.279994,0.275765,1.575636,53.083067,2.579463,1.667778,0.84478,1,0,0,0
99907,2.534995,2.308533,2.108605,107.757316,2.330251,61.279994,1.342378,1.575636,43.484412,3.572938,1.667778,0.84478,1,0,0,0


## Privacy Attack Simulation

The steps are as follows:

- Split the data three ways:
    - Marketer training data
    - Adversary training data
    - Outside data
- Train Marketer and Adversary synthesis models
- Compute predictions for distribution membership for outside data and compute empirical epsilon
- Repeat the above steps many times (100 iterations in Ponte et al. 2024)

Function to split into train and test sets while ensuring that the training data has an even number of rows.

In [67]:
def train_test_split_even(X, train_size, random_state=None):
    # Split the data normally
    X_train, X_test = train_test_split(
        X, train_size=train_size, random_state=random_state
    )
    
    # If train set has odd number of rows
    if len(X_train) % 2 != 0:
        # Move the last row from train to test
        X_test = pd.concat([X_test, X_train[-1:]], axis=0)
        X_train = X_train[:-1]
    
    return X_train, X_test

Parameters for synthesis models.

In [68]:
# synthesis steps
# written as a list of tuples (features, model)
synthesis_steps = [
    (["f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9", "f10", "f11"], 'gmm'),
    ('treatment', 'multinomial'),
    ('exposure', 'multinomial'),
    ('visit', 'multinomial'),
    ('conversion', 'multinomial')
]

In [69]:
# parameter bounds
param_bounds = {
    'gmm': {
        'num_components': (10, 50.99),
    },
    'multinomial': {
        'treatment': {'C': (0.001, 3)},
        'exposure': {'C': (0.001, 3)},
        'visit': {'C': (0.001, 3)},
        'conversion': {'C': (0.001, 3)}
    }
}

In [70]:
number_synthetic_datasets = 10
num_iter_optimization = 5
num_init_optimization = 2
random_states = [1006, 428]
poly_degree_mnl = 2
poly_degree_pmse = 2
interaction_only = True
gmm_n_init = 3
covariance_type = "diag"

#### Set Everything Up in a Loop

In [71]:
num_obs = [3000, 30000]
num_simulations = 2
epsilons = {}
seed = 42

Order training data variables based on synthesis order.

In [72]:
var_names = []
for item in synthesis_steps:
    name = item[0]
    if type(name) == list:
        for i in name:
            var_names.append(i)
    else:
        var_names.append(name)


In [73]:
train_data = train_data[var_names]
train_data

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,exposure,visit,conversion
0,2.954807,2.368327,2.178833,1.480282,2.330251,20.349663,0.000205,2.100938,48.158740,2.579463,1.667778,0.84478,1,0,0,0
1,3.100481,2.308533,2.105887,107.757316,2.330251,61.279994,0.275765,1.575636,53.083067,2.579463,1.667778,0.84478,1,0,0,0
2,2.945821,2.308533,2.139045,9.914112,2.330251,61.279994,0.003786,1.575636,46.910867,3.486706,1.667778,0.84478,0,0,0,0
3,3.177230,2.308533,2.105887,107.757316,2.330251,61.279994,0.089715,1.575636,53.083067,2.579463,1.667778,0.84478,0,0,0,0
4,3.274234,2.308533,2.105887,107.757316,2.330251,61.279994,0.000003,1.575636,53.083067,2.579463,1.667778,0.84478,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99904,3.037003,2.308533,2.184994,107.757316,2.330251,61.279994,0.275765,1.575636,47.003884,2.579463,1.667778,0.84478,1,0,0,0
99905,3.286020,2.308533,2.105887,107.757316,2.330251,61.279994,0.275765,1.575636,53.083067,2.579463,1.667778,0.84478,1,0,0,0
99906,3.285632,2.308533,2.105887,107.757316,2.330251,61.279994,0.275765,1.575636,53.083067,2.579463,1.667778,0.84478,1,0,0,0
99907,2.534995,2.308533,2.108605,107.757316,2.330251,61.279994,1.342378,1.575636,43.484412,3.572938,1.667778,0.84478,1,0,0,0


In [74]:
for n in num_obs:

    current_data_sample, _ = train_test_split(train_data, train_size = n, stratify = train_data['conversion'], random_state=seed)

    epsilons[n] = []

    for i in range(num_simulations):

        # split data
        internal_data, external_data = train_test_split_even(current_data_sample, train_size=0.67, random_state=seed+i)
        marketer_train, adversary_train = train_test_split(internal_data, train_size=0.5)

        # marketer synthesis
        marketer_optimization_results = [optimize_models(train_data=marketer_train, 
                                                         number_synthetic_datasets=number_synthetic_datasets,
                                                         synthesis_steps=synthesis_steps,
                                                         param_bounds=param_bounds,
                                                         poly_degree_mnl=poly_degree_mnl,
                                                         poly_degree_pmse=poly_degree_pmse,
                                                         interaction_only=interaction_only,
                                                         covariance_type=covariance_type,
                                                         gmm_n_init=gmm_n_init,
                                                         random_state=r,
                                                         num_iter_optimization=num_iter_optimization,
                                                         num_init_optimization=num_init_optimization) for r in random_states]

        # adversary synthesis
        adversary_optimization_results = [optimize_models(train_data=adversary_train, 
                                                          number_synthetic_datasets=number_synthetic_datasets,
                                                          synthesis_steps=synthesis_steps,
                                                          param_bounds=param_bounds,
                                                          poly_degree_mnl=poly_degree_mnl,
                                                          poly_degree_pmse=poly_degree_pmse,
                                                          interaction_only=interaction_only,
                                                          covariance_type=covariance_type,
                                                          gmm_n_init=gmm_n_init,
                                                          random_state=r,
                                                          num_iter_optimization=num_iter_optimization,
                                                          num_init_optimization=num_init_optimization) for r in random_states]

        # store best params
        best_marketer_params = marketer_optimization_results[np.argmin([x['best_score'] for x in marketer_optimization_results])]['best_params']
        best_adversary_params = adversary_optimization_results[np.argmin([x['best_score'] for x in adversary_optimization_results])]['best_params']

        # train and generate with best params
        marketer_ratios, marketer_sXs = perform_synthesis(train_data=marketer_train,
                                                          number_synthetic_datasets=2,
                                                          poly_degree_mnl=poly_degree_mnl,
                                                          poly_degree_pmse=poly_degree_pmse,
                                                          interaction_only=interaction_only,
                                                          covariance_type=covariance_type,
                                                          gmm_n_init=gmm_n_init,
                                                          synthesis_steps=synthesis_steps,
                                                          param_values=best_marketer_params)

        adversary_ratios, adversary_sXs = perform_synthesis(train_data=adversary_train,
                                                            number_synthetic_datasets=2,
                                                            poly_degree_mnl=poly_degree_mnl,
                                                            poly_degree_pmse=poly_degree_pmse,
                                                            interaction_only=interaction_only,
                                                            covariance_type=covariance_type,
                                                            gmm_n_init=gmm_n_init,
                                                            synthesis_steps=synthesis_steps,
                                                            param_values=best_adversary_params)

        marketer_synthetic = marketer_sXs[0]
        adversary_synthetic = adversary_sXs[0]

        ### below code borrowed from Ponte et al. (2024)
        # to prevent a naive model
        N = len(marketer_train)/10

        # step 1, 2 from paper
        params = {"bandwidth": np.logspace(-1, 1, 20)} # vary the bandwith
        grid_marketer = GridSearchCV(KernelDensity(), params, n_jobs = -1) # cross validate for bandwiths
        grid_marketer.fit(marketer_train) # estimate pdf from train data.
        marketer_kde = grid_marketer.best_estimator_ # get best estimator

        grid_adversary = GridSearchCV(KernelDensity(), params, n_jobs = -1) # cross validate (CV)
        grid_adversary.fit(adversary_train) # estimate pdf from adversary data
        kde_adversary = grid_adversary.best_estimator_ # get best estimator from CV

        density_marketer = marketer_kde.score_samples(marketer_train) # score train examples from train on pdf_train
        density_adversary = kde_adversary.score_samples(adversary_train) # score train examples from train on pdf_adversary
        TPR = sum(density_marketer > density_adversary)/len(density_marketer) # calculate TPR

        density_marketer_new = marketer_kde.score_samples(external_data) # score eval_outside examples on train density
        density_adversary_new = kde_adversary.score_samples(external_data) # score eval_outside examples on adversary density
        FPR = sum(density_marketer_new > density_adversary_new)/len(density_marketer_new) # calculate FPR
        TNR = 1 - FPR
        FNR = 1 - TPR
        print(TPR)
        print(FPR)
        print(TNR)
        print(FNR)
        epsilons[n].append(max(math.log((1 - (1/N) - FPR)/FNR), math.log((1 - (1/N) - FNR)/FPR))) # append resulting epsilon to epsilons

    
    

|   iter    |  target   |    x0     |    x1     |    x2     |    x3     |    x4     |
-------------------------------------------------------------------------------------


c:\Users\camer\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means th

| 1         | -0.461810 | 40.460790 | 0.6800465 | 1.9279589 | 2.6471840 | 2.2223556 |


c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter 

| 2         | -0.416391 | 39.564593 | 0.8867481 | 1.5770236 | 0.5221198 | 2.4360656 |


...

c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter 

| 7         | -0.416634 | 14.048792 | 2.0893390 | 1.8893428 | 1.6912911 | 0.1487642 |


c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\camer\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to ha

0.0
0.07777777777777778
0.9222222222222223
1.0


ValueError: math domain error

In [23]:
np.max(epsilons)

0.34385109043336326